# Sesión 03: Preparación de datos
*"Es un error capital teorizar antes de tener datos. Sin darse cuenta, uno empieza a deformar los hechos para que se ajusten a las teorías, en lugar de ajustar las teorías a los hechos." Arthur Conan Doyle en "Escándalo en Bohemia".*

## 3.1 Selección de filas y columnas
El archivo "hoteles-vienna.xlsx" contiene información de hoteles con habitaciones disponibles para cierto fin de semana. Aunque tenemos varias variables, solamente utilizaremos las siguientes:
- hotel_id: es un identificador que reemplaza el nombre del hotel por razones de confidencialidad.
- accommodationtype: tipo de hospedaje
- price: precio.
- center1distance: distancia al centro
- starrating: calificación de 1 a 5 estrellas (más es mejor). 
- guestreviewsrating: calificación promedio otorgada por los clientes

In [ ]:
# Importa la biblioteca de pandas
import pandas as pd

In [ ]:
variables = ["hotel_id", "accommodationtype", "price", "center1distance",
             "starrating", "guestreviewsrating"]

In [ ]:
# Carga el archivo 'data/b05a_hoteles-viena.xlsx'
df = pd.read_excel('data/b05a_hoteles-viena.xlsx', usecols=variables)

La base de datos abarca diferentes tipos de hospedajes. Mediante `df['A'].value_counts()` podemos consultar qué categorías y frecuencias tiene una variable 'A' en un dataframe llamado 'df'.

In [ ]:
# Obtén la frecuencia por tipo de hospedaje
df['accommodationtype'].value_counts()

Selecciona solamente las filas que corresponden a hoteles. Por ejemplo: 
`df = df[df['accommodationtype'] == 'Hotel']`

In [ ]:
# Selecciona las filas correspondientes a hoteles
df = df[df['accommodationtype'] == 'Hotel']

In [ ]:
# Consulta la información de las variables con df.info()
df.info()

In [ ]:
# Revisa las primeras cinco filas
df.head()

## 3.2 Datos duplicados
En algunas ocasiones puede haber datos duplicados en nuestra base de datos. Para visualizar los datos duplicados podemos usar *duplicated()* de la siguiente manera:
`df[df.duplicated()]`. Si nos interesa en la variable "A" en particular entonces es `df[df['A'].duplicated()]`

In [ ]:
# Identifica los datos duplicados en todas las variables 
df[df.duplicated()]

Una opción para eliminar datos duplicados es usar *drop_duplicates()*. En nuestro ejemplo sería: `df.drop_duplicates()`. Esto sólo elimina observaciones filas duplicadas en todas las variables. De manera predeterminada, solamente conserva la primera fila de ellas.  
En algunas ocasiones necesitaremos eliminar observaciones duplicadas en solamente algunas variables. En ese caso se puede agregar un listado con el argumento *subset*. En nuestro ejemplo: `df = df.drop_duplicates(subset=['hotel_id'])`

In [ ]:
# Elimina las observaciones duplicadas
df = df.drop_duplicates(subset=['hotel_id'])

## 3.3 Datos perdidos
En el caso de datos perdidos es importante determinar si se pueden considerar aleatorios o si existe un patrón que pueda afectar los resultados. En una encuesta, por ejemplo, es posible que las personas que prefieran no mencionar su ingreso tengan ciertas características, o bien, que las personas tiendan a no responder a preguntas sobre cierto tipo de comportamiento que no es socialmente aceptable.

Es recomendable realizar pruebas estadísticas para verificar si los datos perdidos pueden ser considerados aleatorios o no. Bajo el procedimiento habitual, se construyen dos grupos, uno de observaciones con datos completos y otro grupo de observaciones con datos perdidos en una variable en particular. Posteriormente, se realizan pruebas para comparar si existen diferencias significativas en los valores promedio de las otras variables

Como una aproximación inicial, el método `info()` puede servir para identificar qué variables tienen valores perdidos.

In [ ]:
# Identifica qué variables tienen valores perdidos con 'info()'
df.info()

Para encontrar las filas con datos perdidos podemos usar `df[df.isna().any(axis=1)]`. El método *isna()* sirve para indicar qué valores son perdidos y el método *any* sirve para indicar qué filas tienen al menos un valor perdido.

In [ ]:
# Identifica qué observaciones tienen datos perdidos 
df[df.isna().any(axis=1)]

*¿Qué hacer con datos perdidos?*  
Los métodos más comunes para tratar con datos perdidos son:
- Borrar filas o columnas con datos perdidos. Si son pocos datos perdidos (menos del 5%) y parecen ser completamente aleatorios, borrar los casos correspondientes es una alternativa.
- Imputación simple: Reemplazar los datos perdidos con un valor calculado a partir de la misma variable. La opción más común y conservadora es utilizar el promedio. Sin embargo, este método es muy criticado debido a que reduce la variabilidad y afecta la estimación de los intervalos de confianza (Treiman, 2009).
- Imputación multivariada: Reemplazar los datos perdidos con un valor calculado a partir de otras variables. En la imputación de datos por medio de la regresión se utilizan otras variables en la base de datos para establecer una ecuación de regresión en la que la variable dependiente es la variable con los datos perdidos. La ecuación de regresión estimada se utiliza para estimar los datos perdidos 


En cuanto a otros métodos de estimación se recomienda la lectura del capítulo “Multiple imputation of missing data” de Treiman (2009).


*Imputación simple o univariada*  
La imputación con la media para una columna 'A' es:  
`df['A'] = df['A'].fillna(df['A'].mean())`  
La imputación con la mediana es  
`df['A'] = df['A'].fillna(df['A'].median())`   
La imputación con la moda es:  
`df['A'] = df['A'].fillna(df['A'].mode().iloc[0])` 

In [ ]:
# Consulta las variables que tienen datos perdidos con 'info()'
df.info()

In [ ]:
# Reemplaza los valores perdidos con la media
df['guestreviewsrating'] = df['guestreviewsrating'].fillna(df['guestreviewsrating'].mean())
df.info()

In [ ]:
# Utilizando 'describe()' obtén las principales medidas numéricas de resumen
df.describe()

Un dataframe lo podemos guardar con `df.to_excel('carpeta/archivo.xlsx', index=False)`

In [ ]:
# Exporta el dataframe depurado con el nombre 'output'
df.to_excel('data/output.xlsx', index=False)

## 3.4 Valores atípicos
Un valor atípico (outlier) es un valor extremo en una o más variables. En el caso de series univariadas, es común utilizar el método del valor z o el criterio del rango intercuartil. Para el caso de datos multivariados generalmente se utilizan criterios basados en la distancia de Mahalanobis y otras técnicas multivariadas. 

*¿Qué hacer con datos atípicos?*
Si el valor extremo es un error de captura o parte de otra población lo recomendable es corregir o borrar el caso o variable. Si el valor extremo es parte de la distribución se debe mantener, pero se recomienda transformar la variable de forma tal que el valor extremo no impacte los resultados (p. ej. transformación logarítmica, transformación de Box-Cox o la recodificación de datos).

*Método del valor z*
En este enfoque un valor atípico es aquel que esté a más de tres desviaciones estándar a partir de la media.

In [ ]:
def atipicos_3s(df, columna):
    media = df[columna].mean()
    desviacion_estandar = df[columna].std()
    limite_superior = media + 3 * desviacion_estandar
    limite_inferior = media - 3 * desviacion_estandar
    df_filtrado = df[(df[columna] <= limite_inferior) | (df[columna] >= limite_superior)]
    return df_filtrado

*(¿Cómo modificarías la función anterior para que arroje los datos depurados SIN valores atípicos?)*

In [ ]:
atipicos_3s(df,'price')

*Método del rango intercuartil*. Otro criterio común es considerar como atípicos los valores que están a más de 1.5 veces el rango intercuartil hacia el extremo a partir del 1er. o 3er. cuartil.

In [ ]:
def atipicos_iqr(df, columna):
    Q1 = df[columna].quantile(0.25)
    Q3 = df[columna].quantile(0.75)
    IQR = Q3 - Q1
    limite_inferior = Q1 - 1.5 * IQR
    limite_superior = Q3 + 1.5 * IQR
    df_filtrado = df[(df[columna] <= limite_inferior) | (df[columna] >= limite_superior)]
    return df_filtrado

In [ ]:
atipicos_iqr(df, 'price')

## Referencias
Treiman, D. J. (2009). *Quantitative data analysis. Doing social research to test ideas*. San Francisco, CA: Jossey-Bass.

## Extra 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.ensemble import IsolationForest

# Crear y ajustar el modelo Isolation Forest
clf = IsolationForest(contamination=0.1, random_state=42)
clf.fit(df[['center1distance', 'price']])
# Predecir los valores atípicos
df['atipicos'] = clf.predict(df[['center1distance', 'price']])
df_outliers = df[df['atipicos'] == -1]

# Visualizar los datos y los valores atípicos detectados
plt.figure(figsize=(5, 3))
plt.scatter(df['center1distance'], df['price'], color='b', label='Datos normales', alpha=0.6)
plt.scatter(df_outliers['center1distance'], df_outliers['price'], color='r', alpha=0.6)
plt.xlabel('center1distance')
plt.ylabel('Price')
plt.title('Detección de valores atípicos con Isolation Forest')
plt.show()